In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-large-en-v1.5')

In [ ]:
import pandas as pd
data = pd.read_csv("./../assets/test.csv",header=None)

In [ ]:
values  = data.values

In [ ]:
for i in range(len(values)):
    values[i][0]=i

In [ ]:
values

In [ ]:
# values

In [ ]:
all_embs =  []
from tqdm import tqdm
for el in tqdm(values):
    embs = model.encode([el[2]], normalize_embeddings=True)
    all_embs.append((el[0],embs))

In [ ]:
all_embs[0][1][0].tolist()

In [ ]:
import chromadb
client = chromadb.Client()

collection = client.create_collection("sample_collection2")


In [ ]:
abc = [el[1][0].tolist() for el in all_embs]

In [ ]:
collection.add(
    embeddings=abc,
    metadatas=[{"headline":el} for el in values[:,1].tolist()],
    ids=[str(el[0]) for el in all_embs]
)

In [ ]:
import random
random_elements = [[el[0],el[2]] for el in random.choices(values,k=30)]

In [ ]:
random_elements

In [ ]:
questions_array = [
    (1763, "What color is the sky?"),
    (3897, "What is the aftermath of Israel's 17-day military offensive in the Gaza Strip, and how is the situation being described?"),
    (6219, "Can you provide details about Jeff Raikes' job interview and career move in the early 1980s?"),
    (5166, "Why is The SCO Group delaying the launch of a website, and what is the subject of ongoing litigation involving the company?"),
    (5718, "What is NASA planning to do with its X-43A hypersonic research aircraft, and what speed is it expected to reach?"),
    (1122, "What impact did Intel Corp's revenue outlook have on shares of personal computer makers, and what concerns are being raised?"),
    (7535, "What is the likely decision of a U.S. bankruptcy court regarding the sale of Russian oil group YUKOS's main production unit?"),
    (6000, "How did US commercial crude oil inventories change in the week ending Nov. 12, according to the Energy Department?"),
    (3219, "Why have UN officials been denied permission to check the safety of migrants deported to Libya from Italy?"),
    (6080, "What is the purpose of reviewing pending criminal cases, and what are they trying to determine?"),
    (2035, "What has caused panic in the bicycling community, and what is the subject of Internet rumors?"),
    (3223, "Can anything stop the next global virus outbreak, and what is being investigated to understand the security system?"),
    (3073, "What is notable about PalmOne's Tungsten T5 in terms of its storage capacity, and who is it designed for?"),
    (2083, "What emotional plea has been issued by the family of a Briton held hostage in Iraq, and how does he view the Arab world?"),
    (6491, "What is the debate among hurricane forecasters, and what are they looking for in tracking maps?"),
    (7151, "Who did the Angels sign as their center fielder, and what is the value and duration of the contract?"),
    (5606, "What is the significance of the T-shirt created by Bob Rose, and what does it speak to regarding the school's football revival?"),
    (5957, "What contract agreement did Detroit Tigers relief pitcher Troy Percival reach with the Tigers?"),
    (5444, "What has the Indian Prime Minister, Dr. Manmohan Singh, stated about the economic reforms in the country?"),
    (4856, "What cost-cutting measures does Credit Suisse First Boston plan to implement, and why?"),
    (3138, "What charges have been brought against a British man, and who does the U.S. authorities contend he conspired with?"),
    (6783, "What changes occurred in the ranking of software infrastructure, hardware upgrades, and security, and when did this happen?"),
    (5684, "Among three digital photography repair programs, what is noteworthy about Adobe Elements?"),
    (1618, "What is IBM's plan for its Power5 line of servers, and what market segment will it target?"),
    (5315, "What is the expectation for the stock related to Celebrex, and what condition could improve its situation?"),
    (5453, "When is the solar sail Cosmos 1 set to be launched, and what is its unique feature?"),
    (1075, "What initiative is Microsoft's Japanese arm taking regarding Windows XP Service Pack 2, and how will it distribute it?"),
    (1543, "What charge does Iran vehemently deny, and what has the IAEA found in its investigation?"),
    (6596, "What free-trade agreement was sealed between Korea and Singapore, and what areas does it cover?"),
    (4498, "What information has been found about the new Treo 650, and what limitation does the Sprint version have regarding Bluetooth?")
]

In [ ]:
instruction = "Represent this sentence for searching relevant passages:"
search_results = []
for q in questions_array:
    query = instruction + " " + q[1]
    query = model.encode(query, normalize_embeddings=True)
    results = collection.query(query_embeddings=[query.tolist()],n_results=8000)
    search_results.append(results["ids"][0].index(str(q[0])))

In [ ]:
results["ids"]

In [ ]:
search_results

In [ ]:
from statistics import mean
mean(search_results)